In [ ]:
!wget http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/mono/OpenSubtitles.raw.en.gz

--2020-06-30 18:42:05--  http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/mono/OpenSubtitles.raw.en.gz
Resolving opus.nlpl.eu (opus.nlpl.eu)... 193.166.25.9
Connecting to opus.nlpl.eu (opus.nlpl.eu)|193.166.25.9|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/mono/en.txt.gz [following]
--2020-06-30 18:42:06--  https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/mono/en.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3663376519 (3.4G) [application/gzip]
Saving to: ‘download.php?f=OpenSubtitles%2Fv2018%2Fmono%2FOpenSubtitles.raw.en.gz’

download.php?f=Open 100%[===================>]   3.41G  17.8MB/s    in 3m 18s  

2020-06-30 18:45:25 (17.6 MB/s) - ‘download.php?f=OpenSubtitles%2Fv2018%2Fmono%2FOpenSubtitles.raw.en.gz

In [ ]:
!mkdir lines

In [ ]:
!gunzip -k '/content/download.php?f=OpenSubtitles%2Fv2018%2Fmono%2FOpenSubtitles.raw.en.gz'

In [ ]:
!split -a 3 -l 100000 '/content/download.php?f=OpenSubtitles%2Fv2018%2Fmono%2FOpenSubtitles.raw.en' lines/lines-

In [ ]:
!ls lines

lines-aaa  lines-ayh  lines-bwo  lines-cuv  lines-dtc  lines-erj  lines-fpq
lines-aab  lines-ayi  lines-bwp  lines-cuw  lines-dtd  lines-erk  lines-fpr
lines-aac  lines-ayj  lines-bwq  lines-cux  lines-dte  lines-erl  lines-fps
lines-aad  lines-ayk  lines-bwr  lines-cuy  lines-dtf  lines-erm  lines-fpt
lines-aae  lines-ayl  lines-bws  lines-cuz  lines-dtg  lines-ern  lines-fpu
lines-aaf  lines-aym  lines-bwt  lines-cva  lines-dth  lines-ero  lines-fpv
lines-aag  lines-ayn  lines-bwu  lines-cvb  lines-dti  lines-erp  lines-fpw
lines-aah  lines-ayo  lines-bwv  lines-cvc  lines-dtj  lines-erq  lines-fpx
lines-aai  lines-ayp  lines-bww  lines-cvd  lines-dtk  lines-err  lines-fpy
lines-aaj  lines-ayq  lines-bwx  lines-cve  lines-dtl  lines-ers  lines-fpz
lines-aak  lines-ayr  lines-bwy  lines-cvf  lines-dtm  lines-ert  lines-fqa
lines-aal  lines-ays  lines-bwz  lines-cvg  lines-dtn  lines-eru  lines-fqb
lines-aam  lines-ayt  lines-bxa  lines-cvh  lines-dto  lines-erv  lines-fqc
lines-aan  l

In [ ]:
import pandas as pd
data = pd.read_csv('/content/cv-unique-has-end-punct-sentences.csv', error_bad_lines=False);
data.head()

,Unnamed: 0,sentence
0,0,He was accorded a State funeral and was buried...
1,1,In American English whilst is considered to be...
2,2,Once again she is seen performing on a compute...
3,3,Hippety Hopper returns in McKimsons Pop Im Pop.
4,4,Today their programs are available on the Inte...


In [ ]:
data_text = data[['sentence']]
data_text['index'] = data_text.index
documents = data_text
documents.head()

,sentence,index
0,He was accorded a State funeral and was buried...,0
1,In American English whilst is considered to be...,1
2,Once again she is seen performing on a compute...,2
3,Hippety Hopper returns in McKimsons Pop Im Pop.,3
4,Today their programs are available on the Inte...,4


In [ ]:
import os
import numpy as np
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
c = 0
nof = 0
speaker = []
responses = []
files = os.listdir('/content/lines')
for file_name in files:
  with open(os.path.join('/content/lines', file_name)) as text:
    for line in text:
      if c%2 == 0:
        speaker.append(line)
      else:
        responses.append(line)
      c+=1
  nof+=1
  if nof == 5:
    break
c = 0
modified_speaker = []
modified_responses = []
for sentence in speaker:
  modified_speaker.append(sentence.replace('\n', ''))
for sentence in responses:
  modified_responses.append(sentence.replace('\n', ''))
speaker = modified_speaker
responses = modified_responses
print(speaker[10:15])
print(responses[10:15])

['And...', "Always hoping he'll make a commitment.", "And maybe it's time stop waiting and move on.", 'I...', 'I think...']
['Always going back to him.', "But now i don't think he ever will.", 'What do you think i should do?', 'I--', 'I think...']


In [ ]:
import pandas as pd
data_text = pd.DataFrame(speaker)
data_text.head()

,0
0,El diablo!
1,El diablo!
2,We'll get some iodine on that hand.
3,"Goodnight, mimi."
4,Just for a minute.


In [ ]:
data_text['index'] = data_text.index
documents = data_text
documents.head()

,0,index
0,El diablo!,0
1,El diablo!,1
2,We'll get some iodine on that hand.,2
3,"Goodnight, mimi.",3
4,Just for a minute.,4


In [ ]:
def lemmatize_stemming(text):
    stemmer = SnowballStemmer('english')
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [ ]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['Is', 'that', 'true?']


 tokenized and lemmatized document: 
['true']


In [ ]:
processed_docs = documents[0].map(preprocess)
processed_docs[:10]

0                 [diablo]
1                 [diablo]
2            [iodin, hand]
3        [goodnight, mimi]
4                  [minut]
5             [look, want]
6             [ask, advic]
7          [yeah, probabl]
8    [think, break, jerri]
9                   [hang]
Name: 0, dtype: object

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 diablo
1 hand
2 iodin
3 goodnight
4 mimi
5 minut
6 look
7 want
8 advic
9 ask
10 probabl


In [ ]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[0]

[]

In [ ]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[]


In [ ]:
print(data_text[0][5], processed_docs[5])

Look, i just wanted to ask, um, ['look', 'want']


In [ ]:
bow_doc_4310 = bow_corpus[0]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

In [ ]:
lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=40, id2word=dictionary, passes=2, workers=2)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.332*"tell" + 0.288*"thank" + 0.046*"chang" + 0.044*"see" + 0.036*"promis" + 0.034*"hide" + 0.021*"couldn" + 0.013*"uncl" + 0.012*"contact" + 0.010*"vote"
Topic: 1 
Words: 0.485*"yeah" + 0.217*"fuck" + 0.046*"understand" + 0.029*"book" + 0.023*"serious" + 0.015*"smell" + 0.014*"boy" + 0.012*"possibl" + 0.011*"appreci" + 0.010*"tight"
Topic: 2 
Words: 0.186*"talk" + 0.142*"fine" + 0.077*"kill" + 0.067*"start" + 0.065*"hell" + 0.045*"second" + 0.037*"open" + 0.028*"child" + 0.017*"seven" + 0.015*"earth"
Topic: 3 
Words: 0.076*"night" + 0.067*"happi" + 0.053*"doctor" + 0.042*"have" + 0.040*"door" + 0.038*"close" + 0.034*"rule" + 0.030*"jesus" + 0.030*"mess" + 0.027*"throw"
Topic: 4 
Words: 0.416*"right" + 0.253*"look" + 0.064*"home" + 0.054*"friend" + 0.050*"kind" + 0.020*"stupid" + 0.015*"moment" + 0.010*"stick" + 0.010*"burn" + 0.009*"sign"
Topic: 5 
Words: 0.283*"go" + 0.141*"littl" + 0.119*"feel" + 0.035*"dude" + 0.033*"walk" + 0.027*"mistak" + 0.021*"stone" + 0.019*

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus_tfidf, texts=documents):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus_tfidf, texts=documents)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

KeyboardInterrupt: ignored